<a href="https://colab.research.google.com/github/faranbutt/Extract_Transform_Load_Ex/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Exercise 1:**
*Perform ETL Job on zip data on*
[Dirty Data](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip)

In [1]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2023-10-10 06:59:07--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: ‘source.zip’

source.zip          100%[===================>]   2.64K  --.-KB/s    in 0s      

2023-10-10 06:59:07 (872 MB/s) - ‘source.zip’ saved [2707/2707]



In [2]:
!unzip /content/source.zip

Archive:  /content/source.zip
  inflating: source3.json            
  inflating: source1.csv             
  inflating: source2.csv             
  inflating: source3.csv             
  inflating: source1.json            
  inflating: source2.json            
  inflating: source1.xml             
  inflating: source2.xml             
  inflating: source3.xml             


In [123]:
import pandas as pd
import glob
from datetime import datetime
import xml.etree.ElementTree as ET

In [124]:
log_file = 'log_data.txt'
target_file = 'output.csv'

In [125]:
def log_data(message):
  timestamp_format = "%Y-%h-%d-%H:%M:%S"
  now = datetime.now()
  timestamp = now.strftime(timestamp_format)
  with open(log_file,'a') as log:
    log.write(f'{timestamp}, {message} \n')

In [126]:
def extract_data_from_csv(csvfile):
  df = pd.read_csv(csvfile)
  return df
def extract_data_from_json(jsonfile):
  df = pd.read_json(jsonfile,lines=True)
  return df


In [127]:
def extract_data_from_xml(xmlfile):
  df = pd.DataFrame(columns=['name','height','weight'])
  tree = ET.parse('/content/source1.xml')
  root = tree.getroot()
  for person in root:
    name = person.find('name').text
    height = float(person.find('height').text)
    weight = float(person.find('weight').text)
    df = pd.concat([df,pd.DataFrame([{"name":name,"height":height,"weight":weight}])],ignore_index=True)
  return df

In [128]:
def extract():
  df = pd.DataFrame(columns=['name','height','weight'])
  #csv data extraction
  for csv_file in glob.glob('*.csv'):
    csv_data = extract_data_from_csv(csv_file)
    df = pd.concat([df,pd.DataFrame(csv_data)],ignore_index=True)

    #json data extraction
  for json_file in glob.glob('*.json'):
    json_data = extract_data_from_json(json_file)
    df = pd.concat([df,json_data],ignore_index=True)

    #xml data extraction
  for xml_file in glob.glob('*.xml'):
    xml_data = extract_data_from_xml(xml_file)
    df = pd.concat([df,xml_data],ignore_index=True)
  return df

In [129]:
def transform(data):
  data['height'] = round(data.height*0.0254,2)
  data['weight'] = round(data.weight * 0.45359237,2)
  return data

In [130]:
def load_data(transformed_data,target_file):
  transformed_data.to_csv(target_file)

In [131]:
log_data('ETL Job Started')
log_data('Extract Phase Started')
extracted_data = extract()
print(extracted_data)
log_data('Extract Phase Ended')

     name  height  weight
0    alex   65.78  112.99
1    ajay   71.52  136.49
2   alice   69.40  153.03
3    ravi   68.22  142.34
4     joe   67.79  144.30
5    alex   65.78  112.99
6    ajay   71.52  136.49
7   alice   69.40  153.03
8    ravi   68.22  142.34
9     joe   67.79  144.30
10   alex   65.78  112.99
11   ajay   71.52  136.49
12  alice   69.40  153.03
13   ravi   68.22  142.34
14    joe   67.79  144.30
15   jack   68.70  123.30
16    tom   69.80  141.49
17  tracy   70.01  136.46
18   john   67.90  112.37
19   jack   68.70  123.30
20    tom   69.80  141.49
21  tracy   70.01  136.46
22   john   67.90  112.37
23   jack   68.70  123.30
24    tom   69.80  141.49
25  tracy   70.01  136.46
26   john   67.90  112.37
27  simon   67.90  112.37
28  jacob   66.78  120.67
29  cindy   66.49  127.45
30   ivan   67.62  114.14
31  simon   67.90  112.37
32  jacob   66.78  120.67
33  cindy   66.49  127.45
34   ivan   67.62  114.14
35  simon   67.90  112.37
36  jacob   66.78  120.67
37  cindy   

In [132]:
log_data("Transform Phase Started")
transformed_data = transform(extracted_data)
print("Tranformed Data")
print(transformed_data)
log_data("Tranform Phase Ended")

Tranformed Data
     name  height  weight
0    alex    1.67   51.25
1    ajay    1.82   61.91
2   alice    1.76   69.41
3    ravi    1.73   64.56
4     joe    1.72   65.45
5    alex    1.67   51.25
6    ajay    1.82   61.91
7   alice    1.76   69.41
8    ravi    1.73   64.56
9     joe    1.72   65.45
10   alex    1.67   51.25
11   ajay    1.82   61.91
12  alice    1.76   69.41
13   ravi    1.73   64.56
14    joe    1.72   65.45
15   jack    1.74   55.93
16    tom    1.77   64.18
17  tracy    1.78   61.90
18   john    1.72   50.97
19   jack    1.74   55.93
20    tom    1.77   64.18
21  tracy    1.78   61.90
22   john    1.72   50.97
23   jack    1.74   55.93
24    tom    1.77   64.18
25  tracy    1.78   61.90
26   john    1.72   50.97
27  simon    1.72   50.97
28  jacob    1.70   54.73
29  cindy    1.69   57.81
30   ivan    1.72   51.77
31  simon    1.72   50.97
32  jacob    1.70   54.73
33  cindy    1.69   57.81
34   ivan    1.72   51.77
35  simon    1.72   50.97
36  jacob    1.70   54

In [133]:
log_data('Load Phase Started')
load_data(transformed_data,target_file)
log_data('Load Phase Ended')
log_data("ETL Job Ended")

In [ ]:
with open('/content/log_data.txt',"r") as log_fil:
  data = log_fil.read()
  print(data)


**Exercise 2:**
*Develop ETL pipeline for zip data on*
[Dirty Data 2](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip)

In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

In [ ]:
!unzip /content/datasource.zip

In [319]:
target_file_2 = 'output2.csv'
log_file_2 = 'log_file2.txt'

In [320]:
def extract_data_from_csv_2(csv_file):
  df = pd.read_csv(csv_file)
  return df
def extract_data_from_json_2(json_file):
  df = pd.read_json(json_file,lines=True)
  return df

In [321]:
def extract_data_from_xml_2(xml_file):
  df = pd.DataFrame(columns=['car_model','year_of_manufacture','price','fuel'])
  tree = ET.parse(xml_file)
  root = tree.getroot()
  for car in root:
    car_model = car.find("car_model").text
    year_of_manufacture = car.find("year_of_manufacture").text
    price = float(car.find("price").text)
    fuel = car.find('fuel').text
    df = pd.concat([df,pd.DataFrame([{"car_model":car_model,"year_of_manufacture":year_of_manufacture,"price":price,"fuel":fuel}])],ignore_index=True)
  return df


In [322]:
def extract_2():
  df = pd.DataFrame(columns=['car_model','year_of_manufacture','price','fuel'])

  #Extract CSV data
  for csv_file in glob.glob('*.csv'):
    csv_data = extract_data_from_csv_2(csv_file)
    df = pd.concat([df,csv_data],ignore_index=True)

  #Extract JSON data
  for json_file in glob.glob('*.json'):
    json_data = extract_data_from_json_2(json_file)
    df = pd.concat([df,json_data],ignore_index=True)

  #Extact XML data
  for xml_file in glob.glob('*.xml'):
    xml_data = extract_data_from_xml_2(xml_file)
    df = pd.concat([df,xml_data],ignore_index=True)

  return(df)

In [323]:
def transform2(data):
  data['price'] = round(data.price,2)
  return data

In [324]:
def load_data_2(transformed_data,target_file_2):
  transformed_data.to_csv(target_file_2)

In [325]:
def log_data_2(message):
  timestamp_format = '%Y-%h-%d-%H:%M:%S'
  now = datetime.now()
  timestamp = now.strftime(timestamp_format)
  with open(log_file_2,'a') as f:
    f.write(f"{timestamp} - {message} \n")

In [326]:
log_data_2("ETL JOB STARTED")
log_data_2("Extract Phase Started")
extracted_data_2 = extract_2()
print(extracted_data_2)
log_data_2("Extract Phase Ended")

        car_model year_of_manufacture         price    fuel
0            ritz                2014   5000.000000  Petrol
1             sx4                2013   7089.552239  Diesel
2            ciaz                2017  10820.895522  Petrol
3         wagon r                2011   4253.731343  Petrol
4           swift                2014   6865.671642  Diesel
..            ...                 ...           ...     ...
85        etios g                2015   5895.522388  Petrol
86  corolla altis                2013   8208.955224  Petrol
87        corolla                2004   2238.805970  Petrol
88  corolla altis                2010   7835.820896  Petrol
89       fortuner                2012  21641.791045  Diesel

[90 rows x 4 columns]


In [327]:
log_data_2("Transform Phase Started")
transformed_data_2 = transform2(extracted_data_2)
print("Transformed Data")
print(transformed_data_2)
log_data_2("Transform Phase Ended")

Transformed Data
        car_model year_of_manufacture     price    fuel
0            ritz                2014   5000.00  Petrol
1             sx4                2013   7089.55  Diesel
2            ciaz                2017  10820.90  Petrol
3         wagon r                2011   4253.73  Petrol
4           swift                2014   6865.67  Diesel
..            ...                 ...       ...     ...
85        etios g                2015   5895.52  Petrol
86  corolla altis                2013   8208.96  Petrol
87        corolla                2004   2238.81  Petrol
88  corolla altis                2010   7835.82  Petrol
89       fortuner                2012  21641.79  Diesel

[90 rows x 4 columns]


In [328]:
log_data_2("Load Phase Started")
load_data_2(transformed_data_2,target_file_2)
log_data_2("Load Phase Ended")
log_data_2("ETL JOB ENDED")

In [329]:
with open('/content/log_file2.txt',"r") as log_fil:
  data = log_fil.read()
  print(data)

2023-Oct-10-12:10:46 - ETL JOB STARTED 
2023-Oct-10-12:10:46 - Extract Phase Started 
2023-Oct-10-12:10:46 - Extract Phase Ended 
2023-Oct-10-12:10:47 - Transform Phase Started 
2023-Oct-10-12:10:47 - Transform Phase Ended 
2023-Oct-10-12:10:48 - Load Phase Started 
2023-Oct-10-12:10:48 - Load Phase Ended 
2023-Oct-10-12:10:48 - ETL JOB ENDED 

